### featureCounts
The next step will use the featurecounts package to assign each read to a "feature" in the GTF file.  
These features run the spectrum of "exons", "gene", "UTR", "start_codon", "stop_codon" etc...  
If you look through typically the third column of your GTF file, you can see all the different types of features.  

For the purposes of the LS pipeline, we will use the default featurecounts setting of mapping to "exons".  
Typically for Diff Expression analysis, the exons are what you want to use.  
There are some papers that do some DEA with introns and other features.  
Light-seq can capture both exons and introns, so the type of analysis you want to do is really up to you!

In [1]:
# Lets take a look at the options in featurecounts
!featurecounts


Version 2.0.1

Usage: featureCounts [options] -a <annotation_file> -o <output_file> input_file1 [input_file2] ... 

## Mandatory arguments:

  -a <string>         Name of an annotation file. GTF/GFF format by default. See
                      -F option for more format information. Inbuilt annotations
                      (SAF format) is available in 'annotation' directory of the
                      package. Gzipped file is also accepted.

  -o <string>         Name of output file including read counts. A separate file
                      including summary statistics of counting results is also
                      included in the output ('<string>.summary'). Both files
                      are in tab delimited format.

  input_file1 [input_file2] ...   A list of SAM or BAM format files. They can be
                      either name or location sorted. If no files provided,
                      <stdin> input is expected. Location-sorted paired-end reads
                      a

### Mandatory requirements
We can see from the options that we have three mandatory arguments.  
We need to provide the path to the GTF file, a path for the summary files and a path to the BAM file of interest.
For mouse and human GTF files. We get our GTF/GFF3 file from genecode, the comprehensive Chr annotation:  
https://www.gencodegenes.org/mouse/

And for the majority of the analysis, we used the vM27 version.  
Ideally you should use the same GTF file for this part of the pipeline that you used to build the index with STAR aligner.  

In [2]:
# Now lets run the featurecounts package on the mapped .bam file from STAR aligner
!featurecounts -a gtf_file/gencode.vM27.annotation.gff3 -o outFiles/LS23A_GeneAssigned -R BAM outFiles/TLS23A_R1_MouseAligned.sortedByCoord.out.bam


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.1

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 1 BAM file                                       ||
||                           o TLS23A_R1_MouseAligned.sortedByCoord.out.bam   ||
||                                                                            ||
||             Output file : LS23A_GeneAssigned                               ||
||                 Summary : LS23A_GeneAssigned.summary          

### Output
You should now have another slightly larger BAM file with a "featureCounts" tag attached to it.  
Note with the default options, we also do not consider multimapping reads.  
However we filtered out multimapping reads at the STAR aligner mapping stage so setting this option to True will not do anything for use at this stage.  

Lets take a look at the two BAM files.

In [3]:
# We will use the pysam module
import pysam

### Pysam
Pysam is the main module we use to read BAM files.  
It has quite an extensive documentation as well and is worth reading.  
https://pysam.readthedocs.io/en/latest/api.html#api

The main thing to understand is what it returns when you read a BAM file.  

In [4]:
# Read the a BAM file, from the STAR aligner
samfile = pysam.AlignmentFile(
    "outFiles/TLS23A_R1_MouseAligned.sortedByCoord.out.bam", "rb"
)

[E::idx_find_and_load] Could not retrieve index file for 'outFiles/TLS23A_R1_MouseAligned.sortedByCoord.out.bam'


#### Note
You will typically get an error saying there is no index file.  
If you want to generate an index file to remove that error, you can run `samtools index`.  
However we do not actually need an index file to iterate over the samfile which is what we are mainly going to do.  
As stated here http://www.htslib.org/doc/samtools-index.html, an index is only needed if you want to retrieve a specific region, like all chr1 maps for example. 

As an example this is mentioned in the `fetch` function for pysam.
```
fetch(self, contig=None, start=None, stop=None, region=None, tid=None, until_eof=False, multiple_iterators=False, reference=None, end=None)
fetch reads aligned in a region.

If there is no index, use until_eof=True.

```
So you would use the `until_eof` flag for BAM files without an index.

There are some other useful pysam functions like `mapped` and `get_index_statistics` that do require an index but will not be discussed here.

In [5]:
#This is the read samfile
samfile

In [6]:
# Note that this is a python Iterator object, meaning you call it with the "next" command.
# Everytime you iterate through this with "next", it will return an "alignmedsegment"
next(samfile)

In [7]:
# We can take a look at the details here
print(next(samfile))

M01675:146:000000000-JNRMM:1:1112:24140:17066_AGGGTA_GGTTGTGGAGGG	0	#0	3270322	255	40M	*	0	0	TTGGAATATGTATATCTATATATCTCTATATATATTTACA	array('B', [38, 38, 37, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 37, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38])	[('NH', 1), ('HI', 1), ('AS', 39), ('nM', 0)]


### Reading the BAM file
The output here should be in the standard SAM/BAM format.  
You have lines correspending to the instrument and cluster info, the alignment scores, the sequence, and the tags.  
All of our tags for "NH" which is number of hits should be 1 since we are only doing unique 

In [8]:
#If you want to call a specific part of the AlignedSegment like so
next(samfile).seq

'TGATATATCTTCACGTTGCCTGCACACACCTTATTTCTGA'

In [9]:
next(samfile).tags

[('NH', 1), ('HI', 1), ('AS', 39), ('nM', 0)]

In [10]:
# Remember once you call a part of the iterator with "next", you cant go back!  
# Lets load it again and count the number of reads in the file.
samfile = pysam.AlignmentFile(
    "outFiles/TLS23A_R1_MouseAligned.sortedByCoord.out.bam", "rb"
)
samcount = 0
for read in samfile:
    samcount+=1
    
print("%s reads" % str(samcount))

[E::idx_find_and_load] Could not retrieve index file for 'outFiles/TLS23A_R1_MouseAligned.sortedByCoord.out.bam'


1731739 reads


If you go back to the featurecounts output, this should be exactly the same amount for Total alignments.  
And if you try to iterate through the samfile again you will get an error since ive already gone through the whole thing.

In [11]:
next(samfile)

StopIteration: 

### Featurecounts BAM file
Featurecounts will add an extra "tag" to the bam file indicating which gene the map got assigned to, lets compare and contrast.

In [12]:
samfile = pysam.AlignmentFile(
    "outFiles/TLS23A_R1_MouseAligned.sortedByCoord.out.bam", "rb"
)
samfile_fc = pysam.AlignmentFile(
    "outFiles/TLS23A_R1_MouseAligned.sortedByCoord.out.bam.featureCounts.bam", "rb"
)

[E::idx_find_and_load] Could not retrieve index file for 'outFiles/TLS23A_R1_MouseAligned.sortedByCoord.out.bam'
[E::idx_find_and_load] Could not retrieve index file for 'outFiles/TLS23A_R1_MouseAligned.sortedByCoord.out.bam.featureCounts.bam'


In [13]:
# Samfile read
print(next(samfile))

M01675:146:000000000-JNRMM:1:1113:14523:9922_AGGGTA_GGGGGGGGAGTG	0	#0	3204065	255	2S38M	*	0	0	ATTTGAACTCAAGACCTTTGGAAGAGCAGTCAATGCTCTT	array('B', [38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38])	[('NH', 1), ('HI', 1), ('AS', 37), ('nM', 0)]


In [14]:
# Featurecounts read
print(next(samfile_fc))

M01675:146:000000000-JNRMM:1:1113:14523:9922_AGGGTA_GGGGGGGGAGTG	0	#0	3204065	255	2S38M	*	0	0	ATTTGAACTCAAGACCTTTGGAAGAGCAGTCAATGCTCTT	array('B', [38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38])	[('NH', 1), ('HI', 1), ('AS', 37), ('nM', 0), ('XS', 'Unassigned_NoFeatures')]


The extra tag here is the "XS" tag, in this case, this read was not assigned and thus was given the tag "unassigned_nofeatures".  
Note that this does not mean that the read doesnt exist in the GTF file.  
Since we are using the "exon" mapping option, this read could be something else, an intron for example.  

In [15]:
# lets try to find a read that did get assigned
for read in samfile_fc:
    if read.get_tag("XS") == "Assigned":
        break

In [16]:
print(read)

M01675:146:000000000-JNRMM:1:1108:12080:20749_AGGGTA_TGGGGTGGAGAA	0	#0	3276206	255	40M	*	0	0	CCTATATGCTAGTACAGATCTTCTGTCCTGGTACTCATTA	array('B', [38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38])	[('NH', 1), ('HI', 1), ('AS', 39), ('nM', 0), ('XS', 'Assigned'), ('XN', 1), ('XT', 'ENSMUSG00000051951.6')]


#### We see that an assigned read has the tag "XS" "Assigned" and "XT", which corresponds to the ENSEMBL Gene ID that it was assigned to.
If you want to get the read tag for the gene and the sequence. You use the `.seq` and `.get_tag` and other functions.  
These are documented in the pysam documentation.

In [17]:
read.seq

'CCTATATGCTAGTACAGATCTTCTGTCCTGGTACTCATTA'

In [18]:
read.get_tag('XT')

'ENSMUSG00000051951.6'

In [19]:
read.mapping_quality

255

In [20]:
read.qual

'GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG'

In [21]:
read.qname

'M01675:146:000000000-JNRMM:1:1108:12080:20749_AGGGTA_TGGGGTGGAGAA'

### Conclusion
Once you have familiarized yourself pysams functionality, you are pretty much halfway done with parsing out the sequencing data for your purposes.  
For example checking for multimapped reads means you look at the "NH" tag.  
Filtering out reads with bad alignment scores etc...  
Of course there should already be packages out there that do most of this stuff.  
For example featurecounts can also do paired end read analysis, you just need to set it as the option.